In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

In [2]:
files = glob.glob("data/*.CSV")

In [3]:
def process_df(df):
    '''
    Converts time stamp column from String datetime to seconds starting from 0 
    and rounds values of selected columns (all but PC Source X Freq and PC Source X Q) 
    to 3 decimals
    
    Parameters
    ----------
    df: pandas df of experiment CSV
    
    
    Returns
    ---------
    date: string of experiment date (mm/dd/yy)
    df: pandas df with converted time column and 3 decimal values for all columns
    '''
    
    # Convert to datetime
    df['Time Stamp'] = pd.to_datetime(df['Time Stamp'])
    
    # Save date
    date = df['Time Stamp'].iloc[0].date().strftime('%m/%d/%Y')

    # Convert datetime to seconds starting from zero
    df['Time Stamp'] = (df['Time Stamp'] - df['Time Stamp'].iloc[0]).dt.total_seconds()
    
    # Round selected columns to three decimals
    rounded_columns = []

    for i in df.columns.values:
        prefix = ['PC Source']
        suffix = ['Q', 'Freq']
        if not all(p in i for p in prefix):
            rounded_columns.append(i)
        else:    
            if not any(s in i for s in suffix):
                rounded_columns.append(i)

    df[rounded_columns] = df[rounded_columns].round(3)

    
    return date, df

# Formula for identifying a steady experiment

- let $\sigma_t$ be the standard deviation which indicates the threshold for stability. We can use 1 as a default value if no specific value is given (default value based on exploratory analysis);
- let $t \in (0,1)$ be the threshold which marks the cutoff point for a settled experiment, e.g. if $t = 0.5$, then the entire last half of the experiment must be stable for an experiment to be considered settled;
- let $\boldsymbol{P}$ be a set containing the parameters of interest;
- let $n$ be the number of time series measurements.

To identify whether an experiment reaches steady state, we iterate through the time series measurements of an experiment from index $j = 1, ..., t \times n$. In each iteration, if all time series measurements $t_j,...,t_n$  for all parameters $p \in \boldsymbol{P}$ has a standard deviation $\sigma \leq \sigma_t$, then the experiment is classified steady as steady from $t_j$ to $t_n$. If the experiment has not been classified as steady until iteration $t \times n$, then the experiment is considered unsteady.

In [12]:
def steady(params, t=0.5, sigma_t=1):

    for i in range(len(files)):

        # Read csv
        df = pd.read_csv(files[i], skiprows=[0,1])

        # Retrieve date and processed df
        date, df = process_df(df)

        rows = len(df.axes[0])
        cols = len(df.axes[1])

        # Find steady state threshold as index
        n_threshold = int(t*rows)

        
        for j in range(0, n_threshold):

            if all(df[p].tail(rows-j).std() <= sigma_t for p in params):
                print(f'Experiment {i+1} steady at time { df["Time Stamp"].iloc[j] }')
                break

In [13]:
sigma_t = 1

t = 0.5

p1 = 'Power Supply 1 Voltage'
p2 = 'Power Supply 3 Voltage'
p3 = 'Power Supply 5 DC Bias'

steady_params = [p1,p2,p3]

steady(steady_params, t, sigma_t)
        

Experiment 1 steady at time 0.0
Experiment 2 steady at time 4.65
Experiment 4 steady at time 0.653
Experiment 5 steady at time 1.06
Experiment 6 steady at time 0.874
Experiment 7 steady at time 1.276
Experiment 9 steady at time 3.378
Experiment 10 steady at time 0.0
Experiment 12 steady at time 2.593


![title](../plots/12experiments.png)